In [18]:
Title: 

ERROR: Error in parse(text = x, srcfile = src): <text>:2:0: unexpected end of input
1: Title:
   ^


In [ ]:
Introduction:
Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
Clearly state the question you will try to answer with your project
Identify and describe the dataset that will be used to answer the question
 

In [ ]:
Heart disease is an umbrella term for a vast range of conditions that affect the heart. Stroke, heart attack, cardiomyopathy, and angina are all 
examples of heart disease. Leading risk factors for heart disease are high blood pressure and cholesterol, obesity, unhealthy diet, and physical inactivity.
Based on a dataset from 1988 consisting of 4 databases, Cleveland, Hungary, Switzerland, and Long Beach V, heart disease could be predicted 
in a patient by looking at 14 attributes. Of these attributes, 5 will be observed in order to successfully determine the presence of heart disease
in a patient. The predictive question being asked is, "By considering -the five attributes-, can heart disease in a patient be predicted?". It is important 
to answer this question because heart disease is evidently fatal, and an early diagnosage increases the chances of recovery and/or 
survival. Though treatment varies based on the type and cause of the disease, changes in lifestyle habits can influence the presence of heart disease significantly.
Recognizing even a possibility of a heart disease early on is important for patients, and thus, this project will allow for that early recognition. 

In [ ]:
Preliminary exploratory data analysis:


Demonstrate that the dataset can be read from the web into R 
Clean and wrangle your data into a tidy format
Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [2]:
library(tidyverse)
library(repr)
library(dplyr)
library(tidymodels)

print("We started by reading in the dataset for heart disease then selecting six variables that we wanted to look at.")
heart_data <- read_csv("project_data/heart.csv")
  
heart_var <- heart_data |>
    select(age, sex, chol, cp, thal, target)
slice(heart_var, 1:5)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

[1] "We started by reading in the dataset for heart disease then selecting six variables that we wanted to look at."


Rows: 1025 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (14): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,chol,cp,thal,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
52,1,212,0,3,0
53,1,203,0,3,0
70,1,174,0,3,0
61,1,203,0,3,0
62,0,294,0,2,0


In [20]:
We split the data with the variables we selected into a training set with 75% of the data, and a test set ith 25% of the data. To see some of the data, we used the slice function to show the first five rows. Then, we checked if there were any rows with missing data which there were not.

ERROR: Error in parse(text = x, srcfile = src): <text>:1:4: unexpected symbol
1: We split
       ^


In [8]:

set.seed(3456)

heart_split <- initial_split(heart_var, prop = 0.75, strata = target)  
heart_train <- training(heart_split)   
heart_test <- testing(heart_split)

slice(heart_train, 1:5)
sum(is.na(heart_train))

age,sex,chol,cp,thal,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
53,1,203,0,3,0
70,1,174,0,3,0
61,1,203,0,3,0
46,1,249,0,3,0
43,0,341,0,3,0


[1] 0

In [ ]:
From here we began to look at each variable and if there were any trends that could be seen. The "target" variable shows whether each person has heart disease which is indicated with a 1, or if they do not have heart disease, 0. 

Methods:
Explain how you will conduct either your data analysis and which variables/columns you will use. 
Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good 
idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize the results

In [ ]:
To predict whether a patient has heart disease, a classifier needs to be taught and trained. This is to increase the accuracy
of the classifier. The data will be split into a training set and a testing set in order to build an efficient and accurate model
by first using only the training set. This will ensure that the test data does not influence the classifier model being trained.
Because the K-nearest neighbors classification algorithm will be used, the value of K to be used must be determined. Performing 
v-fold cross-validation will determine this. The K value that yields roughly the optimal accuracy, and the value that does not drastically 
change the estimated accuracy when decreasing or increasing K by a small amount will be the best option. Now, the K-nearest neighbors 
classifier object will be used to predict the class labels of the test set and predict whether there is a presence of heart disease in the 
patient. The patient will be evaluated with an integer value of 0 or 1 in the "target" field; 0 = no disease; 1 = disease.
The variables considered in the data analysis will be: ...

In [ ]:
Expected outcomes and significance:
What do you expect to find?
What impact could such findings have?
What future questions could this lead to?